# Buisness Understanding
**Stakeholder:** 
* Medical Device Company

**Problem:** 
* The major issue is being unable to produce effective monitoring and treatment technologies for myocardial infarction (MI) survivors. Being able to predict myocardial complications is essential for advancing technologies in this background. MI can occur without complications or with complications that do not worsen the long-term prognosis. However about half of the patients in the acute and subacute periods have complications that lead to worsening of the disease and even death. Predicting complications of myocardial infarction is importnat in order to carry out the necessary preventive measures in upcoming developing medical devices that will keep those complication in mind.


# Data Understanding

**Source:**
* [University of California Irvine (UCI) Machine Learning Repositories](https://archive.ics.uci.edu/) 

**Dataset:**
* [Myocardial Infarction Complications](https://archive.ics.uci.edu/dataset/579/myocardial+infarction+complications)
    * 1700 rows, 124 columns 
    * 8 potential Target columns, 116 Feature columns
    * All the column names and defintions can be found in the [Column Descriptions](../data/column_descriptions.md) file

**Targets (Myocardial Infarction Complications):**
* FIBR_PREDS (Atrial Fibrillation) 
    * Life-threatening, irregular heartbeat caused by fast and irregular contractions in the upper chambers of the heart. Prevents the heart from pumping blood to the lower chamber of the heart
* PREDS_TAH (Supraventricular Tachycardia)
    * Irregular, fast, or erratic heartbeat that affects the heart's upper chambers. Its not usually serious and does not cause sudden death, heart damage, or heart attacks. In extreme cases that can result but its very unlikely.
* JELUD_TAH (Ventricular Tachycardia)
    * Irregular, fast, or erratic heartbeat that affects the heart's lower chambers. Can become life-threatening if the episode lasts longer than a few seconds also known as a sustained Ventricular Tachycardia. 
* FIBR_JELUD (Ventricular Fibrillation)
    * Life-threatening, irregular heartbeat that affects the heart's ventricles. The lower heart chambers contract rapidly and in an uncoordinated manner. As a result, prevents the heart from pumping blood to the rest of the body.
* A_V_BLOK (Third-degree AV block)
    * Medical condition that occurs when there is a complete loss of communcation between the heart's atria and ventricles. In other words, electrical signals cannot pass from the atria to the ventricles.   
* OTEK_LANC (Pulmonary edema)
    * Life-threatening condition caused by fluid build up in the lungs. This fluid collects in the air sacs in the lungs, making it difficult to breathe.
* RAZRIV (Myocardial rupture)
    * Tear in the heart that occurs after a heart attack. Is life-threatening however is a rare complication of a heart attack.
* DRESSLER (Dressler Syndrome)
    * Inflammation of the sac (pericardium) surrounding the heart. Immune system response due to damage to heart tissue or the sac itself.

**Data Types:**

For this dataset almost all columns have already been numerically encoded for nominal and ordinal columns. There are columns that are numeric that represent realtime values that were taken. These need to be scaled so they don't dramatically impact the modeling results.

Kept Continous Data: 
* ALT_BLOOD (Serum AIAT content)
* AST_BLOOD (Serum AsAT Content)
* L_BLOOD (White Blood Cell Count)
* AGE (Age of patient)

# Model Preparation

Lets split data into a train-test split. 

In [72]:
import pandas as pd
df = pd.read_csv("../data/data_cleaned.csv")


In [73]:
df.head()

,ID,AGE,SEX,INF_ANAM,STENOK_AN,FK_STENOK,IBS_POST,GB,SIM_GIPERT,DLIT_AG,...,PREDS_TAH,JELUD_TAH,FIBR_JELUD,A_V_BLOK,OTEK_LANCRAZRIV,DRESSLER,ZSN,REC_IM,P_IM_STEN,LET_IS
0,2,55,1,1,0.0,0.0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,6,64,1,0,1.0,2.0,1,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,7,70,1,1,1.0,2.0,1,2,0,7.0,...,0,0,0,0,0,0,0,1,0,0
3,10,77,0,2,0.0,0.0,0,3,0,6.0,...,0,0,0,0,0,0,0,1,0,0
4,11,71,1,0,0.0,0.0,0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [74]:
# Import libraries
from sklearn.model_selection import train_test_split

# Define X, y 
X = df.iloc[:, :-12] # Features
y = df.iloc[:, -12:] # Targets

# Perform split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=45)

Lets standardize our contnous columns in our training data

In [75]:
# Define our numeric columns that need to be normalized
numeric_features = [
    "ALT_BLOOD",
    "AST_BLOOD",
    "L_BLOOD",
    "AGE"
]

X_train_numeric = X_train[numeric_features].copy()
X_test_numeric = X_test[numeric_features].copy()

In [76]:
from sklearn.preprocessing import StandardScaler

# Instantiate scaler 
scaler = StandardScaler()

# Fit Scaler to continous training data
scaler.fit(X_train_numeric)

# Concatenate training data and testing data respectively
X_train_scaled = pd.DataFrame(
    scaler.transform(X_train_numeric), 
    index=X_train_numeric.index,
    columns=X_train_numeric.columns
)

X_test_scaled = pd.DataFrame(
    scaler.transform(X_test_numeric), 
    index=X_test_numeric.index,
    columns=X_test_numeric.columns
)

In [77]:
# Concatenate the scaled data with the original training data and drop the columns that are not scaled

# Drop old columns
X_train.drop(columns=numeric_features, inplace=True)
X_test.drop(columns=numeric_features, inplace=True)

# Concatenate the scaled data with the original training data 
X_train_full = pd.concat([X_train, X_train_scaled], axis=1)
X_test_full = pd.concat([X_test, X_test_scaled], axis=1)

c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


# Fitting Baseline Models

Create baseline moedels with no modifications to there performances. The well known fact about this dataset is that there are class imbalances so the baseline moedels should perform poorly.

In [80]:
# Import libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report, log_loss

In [81]:
# Create Pipeline 
# Define the pipelines for each algorithm Baseline
logreg_pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(LogisticRegression(fit_intercept=False, C=1e12, solver='liblinear')))
])

rf_pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(RandomForestClassifier()))
])

knn_pipeline = Pipeline([
    ('classifier', MultiOutputClassifier(KNeighborsClassifier()))
])

# Fit and evaluate the each baseline pipeline
pipelines = [logreg_pipeline, rf_pipeline, knn_pipeline]

# Fit and evaluate each pipeline
for i, pipeline in enumerate([logreg_pipeline, rf_pipeline, knn_pipeline]):
    print(f"Training {pipelines[i]}...")
    
    # Fit the pipeline on the training data
    pipeline.fit(X_train_full, y_train)

    print(f"Evaluating {pipelines[i]}...")
    
    # Make predictions on the test data
    predictions = pipeline.predict(X_test_full)

    # Evaluate the pipeline's performance
    loss = log_loss(y_test, predictions)
    report = classification_report(y_test, predictions, target_names=y.columns)

    print(f"Log Loss:{loss}")
    print("\n" + "=" * 40 + "\n")
    print(f"{pipelines[i]} Classification Report:\n", report)
    print("\n" + "=" * 40 + "\n")

Training Pipeline(steps=[('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1000000000000.0,
                                                                    fit_intercept=False,
                                                                    solver='liblinear')))])...
Evaluating Pipeline(steps=[('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1000000000000.0,
                                                                    fit_intercept=False,
                                                                    solver='liblinear')))])...
Log Loss:13.610393938159273


Pipeline(steps=[('classifier',
                 MultiOutputClassifier(estimator=LogisticRegression(C=1000000000000.0,
                                                                    fit_intercept=False,
                                                                    solver='liblinear')))]) Classification Report:
                  precis

c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluating Pipeline(steps=[('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])...
Log Loss:4.213533347034935


Pipeline(steps=[('classifier',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))]) Classification Report:
                  precision    recall  f1-score   support

      TRENT_S_n       0.33      0.04      0.07        72
     FIBR_PREDS       1.00      0.03      0.05        36
      PREDS_TAH       0.00      0.00      0.00         6
      JELUD_TAH       0.00      0.00      0.00        10
     FIBR_JELUD       0.00      0.00      0.00        13
       A_V_BLOK       0.00      0.00      0.00         8
OTEK_LANCRAZRIV       0.00      0.00      0.00        27
       DRESSLER       0.00      0.00      0.00        13
            ZSN       0.00      0.00      0.00         9
         REC_IM       0.94      0.21      0.34        73
      P_IM_STEN       0.00      0.00      0.00        33
         LET_IS       0.00    

c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\delga\anaconda3\envs\learn-env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\delga\ana

As expected we did poorly. In every model we have very low recall (Sensitivity). This suggests our models can't predict actual positive cases. The log loss for all models are very high. 

The next step is to fix the class imbalance in our data because this is significantly influencing prediction capabilities. 

# Class Imbalance

We cant oversample because some feature are just so rare that it can be 1 or 2. Undersampling is a possibility however it runs the risk of underfitting. 

We can use Synthetic Minority Oversampling (SMOTE)